In [4]:
from requests_html import HTMLSession
import pandas as pd
import time
import re
session = HTMLSession()

In [5]:
base = "https://coinmarketcap.com"

### Scraping the rows

Get every historical link that coinmarketcap provides and put them in a list:

In [6]:
r = session.get(base + "/historical/")
p = re.compile(r'/historical/\d{8}/')
links = [ s for s in r.html.links if p.match(s)]

The global variable `state` is used to collect a list of rows for each page indexed by the link of each page. It is initialized with `False` for each page to signify that the page still has to be scraped. 

In [109]:
state = {link: False for link in links}

This is how many pages we (still) have to scrape:

In [117]:
len([key for (key, value) in state.items() if not value])

0

This tries to scrape the websites and sleeps 6 seconds after each requests. Moreover it sleeps 30 seconds if it encounters and `IndexError` which happened whenever the request was blocked. After it was blocked 10 times it stops; use the cell above to check whether there are still pages remaining and then rerun.

In [118]:
for _ in range(10):
    try:
        for link in (key for (key, value) in state.items() if not value):
            print(link)
            r = session.get(base + link)
            table = r.html.find("table")[2]
            table_content = table.lxml[0]
            tbody = table_content[1]
            state[link] = [[
                pd.to_datetime(link[12:-1]),
                int(trow[0][0].text), 
                trow[1][0][1].text,
                trow[2][0].text,
                int(trow[3][0].text[1:].replace(",","")),
                float(trow[4][0][0].text[1:].replace(",","")),
                int(trow[5][0].text.split(" ")[0].replace(",","")),
                trow[1][0][0].attrib["src"] if "src" in trow[1][0][0].attrib else "",
            ] for trow in tbody]
            time.sleep(6)
    except IndexError:
        print("Index Error")
        time.sleep(30)
print("Stopped")

Stopped


Sum over the lists of rows in `state`, i.e. concatenate them, and put them into a big dataframe:

In [119]:
df = pd.DataFrame(sum((value for key, value in state.items() if value), []), 
    columns=[
        "date",
        "rank", 
        "name",
        "symbol",
        "market-cap",
        "price",
        "circulating-supply",
        "image"
    ])
df

,date,rank,name,symbol,market-cap,price,circulating-supply,image
0,2016-03-06,1,Bitcoin,BTC,6232667417,407.710000,15287125,https://s2.coinmarketcap.com/static/img/coins/...
1,2016-03-06,2,Ethereum,ETH,882886701,11.380000,77562550,https://s2.coinmarketcap.com/static/img/coins/...
2,2016-03-06,3,XRP,XRP,267938591,0.007860,34090841338,https://s2.coinmarketcap.com/static/img/coins/...
3,2016-03-06,4,Litecoin,LTC,144323374,3.220000,44798026,https://s2.coinmarketcap.com/static/img/coins/...
4,2016-03-06,5,MaidSafeCoin,MAID,47222405,0.104347,452552412,https://s2.coinmarketcap.com/static/img/coins/...
...,...,...,...,...,...,...,...,...
73042,2019-05-05,196,Einsteinium,EMC2,18092012,0.082752,218628620,
73043,2019-05-05,197,Gnosis,GNO,18041846,16.330000,1104590,
73044,2019-05-05,198,Ankr,ANKR,17958625,0.008123,2210758610,
73045,2019-05-05,199,Odyssey,OCN,17694306,0.002362,7491360438,


A double index can also be used for further analysis:

In [123]:
df.set_index(["date", "rank"])

name symbol  market-cap       price  \
date       rank                                                        
2016-03-06 1                  Bitcoin    BTC  6232667417  407.710000   
           2                 Ethereum    ETH   882886701   11.380000   
           3                      XRP    XRP   267938591    0.007860   
           4                 Litecoin    LTC   144323374    3.220000   
           5             MaidSafeCoin   MAID    47222405    0.104347   
...                               ...    ...         ...         ...   
2019-05-05 196            Einsteinium   EMC2    18092012    0.082752   
           197                 Gnosis    GNO    18041846   16.330000   
           198                   Ankr   ANKR    17958625    0.008123   
           199                Odyssey    OCN    17694306    0.002362   
           200   Moeda Loyalty Points    MDA    17636839    0.898514   

                 circulating-supply  \
date       rank                       
2016-03-06 1               15287125   
           2               77562550   
           3            34090841338   
           4               44798026   
           5              452552412   
...                             ...   
2019-05-05 196            218628620   
           197              1104590   
           198           2210758610   
           199           7491360438   
           200             19628888   

                                                             image  
date       rank                                                     
2016-03-06 1     https://s2.coinmarketcap.com/static/img/coins/...  
           2     https://s2.coinmarketcap.com/static/img/coins/...  
           3     https://s2.coinmarketcap.com/static/img/coins/...  
           4     https://s2.coinmarketcap.com/static/img/coins/...  
           5     https://s2.coinmarketcap.com/static/img/coins/...  
...                                                            ...  
2019-05-05 196                                                      
           197                                                      
           198                                                      
           199                                                      
           200                                                      

[73047 rows x 6 columns]

Save the dataframe into a csv:

In [124]:
df.sort_values("date").to_csv("cryptocurrency-data.csv", index=False)

### Downloading images

List containing the image urls that have been donwloaded already:

In [170]:
downloaded_already = []

Iterate over all rows which have a download url and download the ones that aren't yet in `downloaded_already`. Download the 64x64 version instead of the 32x32 version.

In [126]:
import urllib.request

In [171]:
for _, row in df.query("image != ''").iterrows():
    if row["image"] not in downloaded_already:
        urllib.request.urlretrieve(row["image"].replace("32","64"), f"assets/{row['name']}.jpg")
        downloaded_already.append(row["image"])

Replace the remote urls in `df2` with the local paths:

In [156]:
df2 = df.copy()
for name in df.query("image != ''").name.unique():
    df2.loc[df.name == name, "image"] = f"assets/{name}.jpg"

In `df2` we have way more rows with images, because:
In `df` a row only had image if it is in the top 10.
Now, it has an image if it *ever* was in the top 10.

In [163]:
df.query("image != ''").name.count()

3997

In [164]:
df2.query("image != ''").name.count()

13900

Top 20 Coins with still no image:

In [185]:
top20noimage = df2.query("rank < 20 and image == ''").name.unique()
top20noimage

array(['Agoras Tokens', 'YbCoin', 'Zetacoin', 'Bitleu',
       'SiliconValleyCoin [old]', 'Peerplays [old]', 'UNUS SED LEO',
       'Qtum', 'Zcash', 'OmiseGO', 'HyperCash', 'VeChain', 'Huobi Token',
       'USD Coin', 'Iconomi', 'NuBits', 'SuperNET', 'HEX', 'Vertcoin',
       'Vechain [Token]', 'FedoraCoin', 'RevolutionVR', 'VeriCoin',
       'CloakCoin', 'BBQCoin', 'Ixcoin', 'BitBar', 'GoldCoin', 'Memecoin',
       'Maker', 'Asiadigicoin', 'VPNCoin', 'Veritaseum',
       'Wrapped Bitcoin', 'BitcoinDark', 'Nxttycoin', 'MonaCoin',
       'Acash Coin', 'Cosmos', 'ICON', 'Maxcoin', 'Rimbit', 'MINDOL',
       'Siacoin', 'TenX', 'Nano', 'Populous', 'Verge', 'Anoncoin',
       'StableCoin', 'GridCoin', 'Rubycoin', 'Gulden', 'Phoenixcoin',
       'Fastcoin', 'Ontology', 'Neutrino', 'FuelCoin', 'Gnosis', '999',
       'ReddCoin', 'NXTventure', 'AmericanCoin', 'Franko', 'Clams',
       'DigiByte', 'Viacoin', 'Decred', 'NetCoin', 'Copperlark',
       'Marinecoin', 'Ardor', 'GameCredits', 'NeuCoi

Download images from the currency's main page:

In [201]:
already_downloaded_images_by_name = []

In [206]:
def download_name(name: str, url_name: str = None):
    print(f"Downloading {name}")
    if not url_name:
        url_name = name.lower().replace(" ", "-").replace("[","").replace("]","")
    r = session.get(f"https://coinmarketcap.com/currencies/{url_name}")
    try:
        url = r.html.find(".cmc-static-icon")[0].lxml[0].attrib["src"]
        urllib.request.urlretrieve(url, f"assets/{name}.jpg")
        already_downloaded_images_by_name.append(name)
    except IndexError:
        print(f"Couldn't find {url_name}")

In [228]:
for name in df2.query("rank <= 30").name.unique():
    if name not in already_downloaded_images_by_name:
        download_name(name)

Couldn't find hyperspace
Couldn't find bytecoin
Couldn't find soundac
Couldn't find lisk
Couldn't find crypto.com-coin
Couldn't find v.systems
Couldn't find augur
Couldn't find golem
Couldn't find getgems
Couldn't find polkadot
Couldn't find dai
Couldn't find swarm
Couldn't find pandacoin
Couldn't find aeternity
Couldn't find jl777hodl
Couldn't find zilliqa
Couldn't find aave
Couldn't find nas
Couldn't find basic-attention-token
Couldn't find unobtanium
Couldn't find mooncoin
Couldn't find pangea-poker
Couldn't find trmb
Couldn't find blockshares
Couldn't find bullion
Couldn't find xencoin
Couldn't find alphacoin
Couldn't find bottlecaps
Couldn't find argentum
Couldn't find dentacoin
Couldn't find paxos-standard
Couldn't find i0coin
Couldn't find datacoin
Couldn't find edc-blockchain
Couldn't find swiscoin
Couldn't find ambercoin
Couldn't find krugercoin
Couldn't find compound
Couldn't find revain
Couldn't find uniswap
Couldn't find razor
Couldn't find vestchain
Couldn't find coinousd


KeyboardInterrupt: 

Three ones have a url that can't be generated from their `name` string:

In [212]:
download_name("Peerplays [old]", "peerplays")
download_name("Vechain [Token]", "vechain")
download_name("999", "999-coin")

Assign newly downloaded asset paths to `df2`:

In [215]:
for name in top20noimage:
    df2.loc[df2.name == name, "image"] = f"assets/{name}.jpg"

Now, there are no images ranked below 20 with no image:

In [219]:
df2.query("rank < 20 and image == ''").name.unique()

array([], dtype=object)

Overwrite the old csv with the new one:

In [220]:
df2.sort_values("date").to_csv("cryptocurrency-data.csv", index=False)